In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku

import ecephys.units as eu

import ecephys

/home/kdriessen/miniconda3/envs/acrdev/lib/python3.10/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
/home/kdriessen/github_t2/kdephys/kdephys/xr/utils.py:4: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [ ]:
import ecephys.units.io as uio

In [ ]:
min_fr, max_fr = (0, float("inf"))
min_depth, max_depth = (0, float("inf"))

ks_dir = Path("path/to/ks/dir")

ksdir = Path('/nvme/sorting/tdt/ACR_14-short-iso-NNXo/ks2_5_nblocks=1_8s-batches/')
sorting = uio.load_sorting_extractor(ksdir, good_only=False, drop_noise=True, selection_intervals={'fr': (min_fr, max_fr), 'depth': (min_depth, max_depth)})

In [ ]:
trn = sorting.get_all_spike_trains()

In [ ]:
sorting.get_unit_ids()

In [ ]:
cluster_info = uio.get_cluster_info(ksdir)
cluster_info = cluster_info[
    cluster_info.cluster_id.isin(sorting.get_unit_ids())
]

In [ ]:
all_spike_times = ecephys.units.get_spike_times_list(sorting, sorting.get_unit_ids())

In [ ]:
sdf = ecephys.units.spikeinterface_sorting_to_dataframe(sorting)

In [ ]:
s2 = sdf.loc[np.logical_and(sdf['t'] > 150, sdf['t'] < 300)]
s2

In [ ]:
quick = sdf.where(sdf.cluster_id == 3).dropna()

In [ ]:
q2 = quick.iloc[0:2000]

In [ ]:
bp = '/Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/'
chan = [2, 5, 7, 12]
lfp = kx.io.get_data(bp, 'LFPo', t1=0, t2=1000, channel=chan)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# duplicate above matplotlib plot in plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=lfp.sel(channel=12).time, y=lfp.sel(channel=12).values, mode='lines', name='LFP'), row=1, col=1)
fig.add_trace(go.Scatter(x=s2.t, y=s2.cluster_id, mode='markers', marker=dict(size=5)), row=2, col=1)
fig

# Unit + Field Potential Inspector

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import plotly.express as px
import ecephys.units as eu

/home/kdriessen/github_t2/kdephys/kdephys/xr/utils.py:4: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [2]:
rec = 'short2-iso'
root = '/nvme/sorting/tdt/' 
ss582 = 'ACR_14-test-short-582/ks2_5_nblocks=1_8s-batches'
ss5104 = 'ACR_14-test-short-5104/ks2_5_nblocks=1_8s-batches'

In [3]:
dp = '/Volumes/opto_loc/Data/ACR_14/short2-iso-LFP_.nc'

In [4]:
fp = xr.load_dataarray(dp)

In [ ]:
d1 = Path(f'/nvme/sorting/tdt/test-short-582/ks2_5_nblocks=1_8s-batches/')
d2 = Path(f'/nvme/sorting/tdt/test-short-5104/ks2_5_nblocks=1_8s-batches/')
ex1 = eu.load_sorting_extractor(d1)
ex2 = eu.load_sorting_extractor(d2)

In [ ]:
ci1 = eu.get_cluster_info(d1)
ci1

In [ ]:
df1 = eu.spikeinterface_sorting_to_dataframe(ex1)
df2 = eu.spikeinterface_sorting_to_dataframe(ex2)

In [ ]:
dti = pd.to_timedelta(df1.t.values, unit='s') + fp.datetime.values.min()
df1['datetime'] = dti

In [ ]:
s2 = sdf.loc[np.logical_and(sdf['t'] > 150, sdf['t'] < 300)]

In [ ]:
def unit_fp_inspector(fp, spk, channel=14, t1=0, t2=120):
    fp = fp.swap_dims({'datetime': 'time'}).sel(time=slice(t1, t2), channel=channel)
    spk = spk.loc[np.logical_and(spk['t'] > t1, spk['t'] < t2)]
    return fp, spk

In [ ]:
f, s = unit_fp_inspector(fp, df1, channel=14, t1=0, t2=120)

In [ ]:
fp.channel.values

In [ ]:

# plotly subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=f.time, y=f.values, name='LFP'), row=1, col=1)
fig.add_trace(go.Scatter(x=s.t, y=s.cluster_id, mode='markers', marker=dict(size=2)), row=2, col=1)

In [6]:
def spectro_data(data, chan, f_range=slice(0, 35)):
    spg = kx.spectral.get_spextrogram(data)
    ax = kx.plots.spectro_plotter(spg, chan=chan, f_range=f_range)
    return ax
    

# Unit Dev

- figure out how to handle differing times between probes

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import acr
import spikeinterface.extractors as se
import spikeinterface.core as si
import kdephys.units as ku
import pandas as pd
import numpy as np
import acr.info_pipeline as aip
import tdt
import acr.units as au

/home/kdriessen/miniconda3/envs/acrdev/lib/python3.10/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
/home/kdriessen/github_t2/kdephys/kdephys/xr/utils.py:4: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [4]:
df.rolling?

Signature:
df.rolling(
    window: 'int | timedelta | BaseOffset | BaseIndexer',
    min_periods: 'int | None' = None,
    center: 'bool_t' = False,
    win_type: 'str | None' = None,
    on: 'str | None' = None,
    axis: 'Axis' = 0,
    closed: 'str | None' = None,
    step: 'int | None' = None,
    method: 'str' = 'single',
) -> 'Window | Rolling'
Docstring:
Provide rolling window calculations.

Parameters
----------
window : int, offset, or BaseIndexer subclass
    Size of the moving window.

    If an integer, the fixed number of observations used for
    each window.

    If an offset, the time period of each window. Each
    window will be a variable sized based on the observations included in
    the time-period. This is only valid for datetimelike indexes.
    To learn more about the offsets & frequency strings, please see `this link
    <https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases>`__.

    If a BaseIndexer subclass, the window bound

In [7]:
hyp['sdpi']

,state,end_time,start_time,duration
0,Wake,2022-09-21 11:15:19.990233375,2022-09-21 09:15:19.999999000,0 days 01:59:59.990234375
0,Wake,2022-09-21 13:15:19.980467750,2022-09-21 11:15:19.990233375,0 days 01:59:59.990234375
0,Wake,2022-09-21 13:26:07.760497047,2022-09-21 13:15:19.980467750,0 days 00:10:47.780029297
1,Transition-to-NREM,2022-09-21 13:26:49.170470191,2022-09-21 13:26:07.760497047,0 days 00:00:41.409973145
2,Wake,2022-09-21 13:32:04.870482398,2022-09-21 13:26:49.170470191,0 days 00:05:15.700012207
...,...,...,...,...
131,Wake,2022-09-21 21:01:19.241209937,2022-09-21 21:01:10.141112281,0 days 00:00:09.100097656
132,NREM,2022-09-21 21:02:43.241209937,2022-09-21 21:01:19.241209937,0 days 00:01:24
133,Transition-to-REM,2022-09-21 21:03:01.341307594,2022-09-21 21:02:43.241209937,0 days 00:00:18.100097656
134,REM,2022-09-21 21:05:05.241209937,2022-09-21 21:03:01.341307594,0 days 00:02:03.899902344


# To be Del'd